<a name="top"></a><img src="./source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [1]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "E:\\GitHub\\SpinalHDL\\Spinal-bootcamp/source/load-spinal.sc"

## Assignement overlap
SpinalHDL will check that no signal assignement completly erases a previous one.   
The following code will throw the following error:

In [3]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a := 66 //Erease the a := 42 :(
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.04 09:49:00
[Progress] at 0.000 : Elaborate components


[Thread-0] WARN net.openhft.affinity.Affinity - Windows JNA-based affinity not usable because it failed to load! Reason: java.lang.NoClassDefFoundError: Could not initialize class com.sun.jna.Native
	at net.openhft.affinity.impl.WindowsJNAAffinity$CLibrary.<clinit>(WindowsJNAAffinity.java:140)
	at net.openhft.affinity.impl.WindowsJNAAffinity.getAffinity(WindowsJNAAffinity.java:61)
	at net.openhft.affinity.impl.WindowsJNAAffinity.<clinit>(WindowsJNAAffinity.java:49)
	at net.openhft.affinity.Affinity.isWindowsJNAAffinityUsable(Affinity.java:87)
	at net.openhft.affinity.Affinity.<clinit>(Affinity.java:45)
	at spinal.sim.JvmThread.run(SimManager.scala:46)

[Thread-0] INFO net.openhft.affinity.Affinity - Using dummy affinity control implementation


[Progress] at 0.163 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (2 errors).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 0.194 : Elaborate components
[Progress] at 0.198 : Checks and transforms


: 

A fix could be:

In [2]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  when(something){
    a := 66
  }
}
showRtl(new TopLevel)

cmd2.sc:4: not found: value something
  when(something){
       ^Compilation Failed

: 

But in the case you really want to override the previous assignement (Yes, it could make sense in some cases), you can do the following:

In [4]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a.allowOverride
  a := 66
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.04 00:02:58
[Progress] at 48.813 : Elaborate components
[Progress] at 48.817 : Checks and transforms
[Progress] at 48.843 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 48.896
// Generator : SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
// Component : TopLevel

`timescale 1ns/1ps 

module TopLevel (
);

  reg        [7:0]    a;
  function [7:0] zz_a(input dummy);
    begin
      zz_a = 8'h2a;
      zz_a = 8'h42;
    end
  endfunction
  wire [7:0] _zz_1;

  assign _zz_1 = zz_a(1'b0);
  always @(*) a = _zz_1;

endmodule



defined class TopLevel

## Clock crossing violation
SpinalHDL will check that every register of your design only depends (through some combinatorial logic) on registers which use the same or a synchronous clock domain. 

The following code will throw error:

In [4]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))   //PlayDev.scala:834
  val regB = clkB(Reg(UInt(8 bits)))   //PlayDev.scala:835

  val tmp = regA + regA                //PlayDev.scala:838
  regB := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.04 09:49:38
[Progress] at 37.446 : Elaborate components
[Progress] at 37.464 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (3 errors).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 37.505 : Elaborate components
[Progress] at 37.508 : Checks and transforms


: 

There are multiple possible fixes:
### crossClockDomain tag
The crossClockDomain tag can be used to say “It’s alright, don’t panic” to SpinalHDL

In [14]:
class TopLevel extends Component {
  val clkaA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkaA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits))).addTag(crossClockDomain)


  val tmp = regA + regA
  regB := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.04 17:59:45
[Progress] at 29444.283 : Elaborate components
[Progress] at 29444.287 : Checks and transforms
[Progress] at 29444.289 : Generate Verilog
[Warning] 3 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 29444.293
// Generator : SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
// Component : TopLevel

`timescale 1ns/1ps 

module TopLevel (
  input               clkA_clk,
  input               clkA_reset,
  input               clkB_clk,
  input               clkB_reset
);

  reg        [7:0]    regA;
  (* async_reg = "true" *) reg        [7:0]    regB;
  wire       [7:0]    tmp;

  assign tmp = (regA + regA);
  always @(posedge clkB_clk) begin
    regB <= tmp;
  end


endmodule



defined class TopLevel

### setSyncronousWith
You can also specify that two clock domains are syncronous together.

In [11]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")
  clkB.setSyncronousWith(clkA)

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits)))


  val tmp = regA + regA
  regB := tmp
}


showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.04 16:33:30
[Progress] at 24269.234 : Elaborate components
[Progress] at 24269.241 : Checks and transforms
[Progress] at 24269.244 : Generate Verilog
[Warning] 3 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 24269.248
// Generator : SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
// Component : TopLevel

`timescale 1ns/1ps 

module TopLevel (
  input               clkA_clk,
  input               clkA_reset,
  input               clkB_clk,
  input               clkB_reset
);

  reg        [7:0]    regA;
  reg        [7:0]    regB;
  wire       [7:0]    tmp;

  assign tmp = (regA + regA);
  always @(posedge clkB_clk) begin
    regB <= tmp;
  end


endmodule



defined class TopLevel

### BufferCC
Signal Bits or Gray-coded Bits can use BufferCC to cross different clockDomain

In [9]:
class syncRead2Write extends Component {
  val io = new Bundle{
    val pushClock, pushRst = in Bool()
    val readPtr = in UInt(8 bits)
  }
  val pushCC = new ClockingArea(ClockDomain(io.pushClock, io.pushRst)) {
    val pushPtrGray = RegNext(toGray(io.readPtr)) init(0)
  }
}
showRtl(new syncRead2Write)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.04 11:45:21
[Progress] at 6980.683 : Elaborate components
[Progress] at 6980.690 : Checks and transforms
[Progress] at 6980.690 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 6980.696
// Generator : SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
// Component : syncRead2Write

`timescale 1ns/1ps 

module syncRead2Write (
  input               io_pushClock,
  input               io_pushRst,
  input      [7:0]    io_readPtr
);

  wire       [7:0]    _zz_pushCC_pushPtrGray;
  reg        [7:0]    pushCC_pushPtrGray;

  assign _zz_pushCC_pushPtrGray = (io_readPtr >>> 1'b1);
  always @(posedge io_pushClock or posedge io_pushRst) begin
    if(io_pushRst) begin
      pushCC_pushPtrGray <= 8'h0;
    end else begin
      pushCC_pushPtrG

defined class syncRead2Write

## Combinational loop
SpinalHDL will check that there are no combinatorial loops in the design.

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := a
  c := 0
}
showRtl(new TopLevel)

A possible fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := 42
  c := 0
}
showRtl(new TopLevel)

### False-positive
It should be said that SpinalHDL’s algorithm to detect combinatorial loops can be pessimistic, and it may give false positives. If it is giving a false positive, you can manualy disable loop checking on one signal of the loop like so:

In [2]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 0
  a(1) := a(0) //False positive because of this line
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.05 14:37:25
[Progress] at 0.000 : Elaborate components


[Thread-0] WARN net.openhft.affinity.Affinity - Windows JNA-based affinity not usable because it failed to load! Reason: java.lang.NoClassDefFoundError: Could not initialize class com.sun.jna.Native
	at net.openhft.affinity.impl.WindowsJNAAffinity$CLibrary.<clinit>(WindowsJNAAffinity.java:140)
	at net.openhft.affinity.impl.WindowsJNAAffinity.getAffinity(WindowsJNAAffinity.java:61)
	at net.openhft.affinity.impl.WindowsJNAAffinity.<clinit>(WindowsJNAAffinity.java:49)
	at net.openhft.affinity.Affinity.isWindowsJNAAffinityUsable(Affinity.java:87)
	at net.openhft.affinity.Affinity.<clinit>(Affinity.java:45)
	at spinal.sim.JvmThread.run(SimManager.scala:46)

[Thread-0] INFO net.openhft.affinity.Affinity - Using dummy affinity control implementation


[Progress] at 0.109 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (2 errors).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 0.142 : Elaborate components
[Progress] at 0.144 : Checks and transforms


: 

could be fixed by :

In [3]:
class TopLevel extends Component {
  val a = UInt(8 bits).noCombLoopCheck
  a := 0
  a(1) := a(0)
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.05 14:37:38
[Progress] at 13.539 : Elaborate components
[Progress] at 13.549 : Checks and transforms
[Progress] at 13.565 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 13.627
// Generator : SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
// Component : TopLevel

`timescale 1ns/1ps 

module TopLevel (
);

  reg        [7:0]    a;

  always @(*) begin
    a = 8'h0;
    a[1] = a[0];
  end


endmodule



defined class TopLevel

It should also be said that assignments such as (a(1) := a(0)) can make some tools like Verilator unhappy. It may be better to use a Vec(Bool, 8) in this case

## Hierarchy violation

SpinalHDL will check that signals are never accessed outside of the current component’s scope.

The following signals can be read inside a component:

- All directionless signals defined in the current component

- All in/out/inout signals of the current component

- All in/out/inout signals of children components

In addition, the following signals can be assigned to inside a component:

- All directionless signals defined in the current component

- All out/inout signals of the current component

- All in/inout signals of children components

If a `HIERARCHY VIOLATION`error appears, it means that one of the above rules was violated.

The following code will throw error:

In [4]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.05 14:39:32
[Progress] at 127.431 : Elaborate components
[Progress] at 127.439 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (3 errors).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 127.445 : Elaborate components
[Progress] at 127.448 : Checks and transforms


: 

A fix could be :

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = out UInt(8 bits) // changed from in to out
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

## Io bundle
SpinalHDL will check that each io bundle contains only in/out/inout signals.

In [6]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = UInt(8 bits)
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.6.4    git head : 598c18959149eb18e5eee5b0aa3eef01ecaa41a1
[Runtime] JVM max memory : 8116.0MiB
[Runtime] Current date : 2022.03.05 14:40:01
[Progress] at 156.111 : Elaborate components
[Progress] at 156.116 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (3 errors).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 156.117 : Elaborate components
[Progress] at 156.119 : Checks and transforms


: 